# モジュール構成について

仮名：Juliaqat

モジュール構成(暫定)

./
- juliaqat.jl : メインモジュール
- Gate.jl : 量子ゲート
- backend_base：メインモジュールとバックエンドで共有する抽象型などを格納

./backends/
- UndirectedGraph.jl : UndirectedGraphModel
- TensorContraction.jl : TensorContractionModel
- その他(上2つも含め、中身はこれから)


インターフェースについて
- 量子回路は mutable stract "qc = QuantumCircuit(n_qubit, n_creg)" として定義
- 量子ゲートも stract "X(target_qubit)" などと定義。全て抽象型 "Gate" のサブクラスとする
- 量子ゲートの適用は "apply!(qc, X(0))" とする。
- 複数の量子ゲートを先にArrayに格納して、"apply!(qc, Array{Gate, 1})"することも可
- backend の選択は "device = get_device("name")" としてdeviceを取得
- 取得する device もstruct。device を引数とした時、その型によって backend の判別を行う。また、backendごとの計算に必要なパラメータの格納にも用いる。
- 量子回路の実行は "result = execute(qc, device)"。device の型に応じて、対応する backend の実行関数を呼び出す。  
Julia では関数名 "run" はBaseクラスが持っているようで、避ける必要がありそう。

- backend に渡すのは QuantumCircuit と GateのArray

# Julia の言語仕様について

Python におけるクラスと等価なものは無い。Julia の struct はメンバ変数を持てるが、関数メソッドを struct に紐付けるようにはなっていない。  
代わりにジェネリック関数が定義できる。  
Python で異なるクラスに属する同名の関数を定義できる代わりに、Julia では異なる引数の型をとる同名の関数を定義する。

言い換えると、Python のクラスによる構成を Julia では型+関数で構成するイメージ。

慣例(明確なコーディング規約はまだ無い模様)
- 変数名は小文字
- 単語の境界はアンダースコア(_)。でもなるべく使用は避ける。
- 型名は大文字で始まり、CamelCase
- 関数名やマクロ名は小文字でアンダースコアなし。
- 引数を破壊的に変更する関数は!をつける(sort!など)。

メモ  
親ディレクトリ中の.jl は、子ディレクトリ中.jlで定義したabstract typeを呼び出せない？ = abstract typeは子から親に渡せない？  


# モジュールインターフェイスの使用例

In [17]:
include("./Juliaqat.jl")
using .Juliaqat

In [24]:
qc = QuantumCircuit(2, 2)
input = Input(20)

Input(20, Complex[0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im], Gate[])

In [25]:
gates = [H(1), H(2), CZ(1, 2), H(1), H(2), CZ(1, 2), H(1), H(2)]
gates = [H(1), H(2), CZ(1, 2), H(1), H(2), CZ(1, 2)]
#gates = [X(1), X(2), X(3)]

6-element Array{Gate,1}:
 H(1)
 H(2)
 CZ(1, 2)
 H(1)
 H(2)
 CZ(1, 2)

In [26]:
apply!(input, gates)

Input(20, Complex[0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im, 0 + 0im], Gate[H(1), H(2), CZ(1, 2), H(1), H(2), CZ(1, 2)])

In [27]:
meas_output = zeros(Int, 20)
device = get_device("UndirectedGraph", meas_output)
typeof(device)

UndirectedGraphModel

In [28]:
state = execute(input, device)
# 開発途中のため返り値はqcとdeviceをそのまま返します

0.49999999999999983 + 0.0im

In [29]:
device.output_basis

20-element Array{Int64,1}:
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0